In [1]:
%matplotlib widget
import numpy as np
import k3d
import matplotlib.pyplot as plt
from functools import reduce
from matplotlib.tri import Triangulation

In [2]:
def getrot(a,b):
    if np.allclose(a,b):
        return np.identity(3)
    else:
        v = np.cross(a, b)
        s = np.linalg.norm(v)
        c = np.dot(a,b)
        vx = np.array([[0,-v[2], v[1]],[v[2], 0, -v[0]], [-v[1], v[0], 0]])
        return np.identity(3) + vx + vx@vx*(1/(1+c))#(1-c)/s**2

def pad3to4(mat):
    m = np.identity(4)
    m[0:3,0:3] = mat
    return m

def norm(v):
    v = np.array(v)
    return v/np.linalg.norm(v)

def vec2rot(a, b):
    if np.allclose(a,b):
        return [0, 1, 0, 0]
    else:
        ez = norm(a)
        n = norm(b)
        ax = np.cross(ez,n)
        ang = np.arccos(ez@n)
        return [ang, ax[0], ax[1], ax[2]]
    
def rot_vec(v, ax, ang):
    if ang==0.0:
        return v
    else:
        return v*np.cos(ang) + np.cross(ax,v)*np.sin(ang) + ax@v*ax*(1. - np.cos(ang))


In [6]:
def disc_coords(n_radii = 5, n_angles = 10, R=1.):
    radii = np.linspace(0., 1, n_radii)
    x, y = [0], [0]
    for r in radii[1:]:
        x.extend([r*np.cos(p) for p in np.linspace(0,2*np.pi,int(2*np.pi*r*n_angles))])
        y.extend([r*np.sin(p) for p in np.linspace(0,2*np.pi,int(2*np.pi*r*n_angles))])
    x = np.array(x)*R
    y = np.array(y)*R
    return x, y

In [96]:
def dbg_shape(lst):
    for l in lst:
        print(l.shape)

In [202]:
class Optic:
    def __init__(self, p=(0,0,0), n=(0,0,1), diam=1.):
        self.p = np.array(p)
        self.n = np.array(norm(n))
        self.r = diam/2
        
    @property  
    def transformation(self):
        #get translation, rotation in k3d format
        return self.p, vec2rot(norm([0,0,1]), self.n)
        
    def plot(self, n_radii = 5, n_angles = 10, **kwargs):
        x, y = disc_coords(n_radii = n_radii, n_angles = n_angles, R=self.r)

        z = np.zeros_like(x)
        indices = Triangulation(x,y).triangles.astype(np.uint32)
        mesh = k3d.mesh(np.vstack([x,y,z]).T, indices, **kwargs)
        t, r = self.transformation
        mesh.transform.rotation = r
        mesh.transform.translation = t
        return mesh  
    
    def _intersect(self, ray, clip=True, p=None):
        #shape of Ray [2xNraysx3]
        x = np.full((ray.shape[1], 3), np.nan)
        
        r, s = ray
        sn = s@self.n
        #prevent ray perpendicular to surface
        msk = np.abs(sn)>np.finfo(np.float32).eps
        t = ((p - r[msk])@self.n)/sn[msk]
        x[msk,:] = r[msk,:] + t[:,None]*s[msk,:]
        
        if clip:
            d = np.linalg.norm(x - p, axis=1)
            #msk[msk] = (d<=self.r)
            x[(d>self.r),:] = np.nan
        return x
    
    def intersect(self, ray, clip=True):
        #for a curved mirror we may have to shift the plane, i.e. p!!
        return self._intersect(ray, clip=clip, p=self.p)
        
    def propagate(self, ray, clip=True):
        rout = np.full_like(ray, np.nan)
        
        q = self.intersect(ray, clip=clip)
        msk = ~np.isnan(q[:,0])
        s = ray[1,msk,:]
        
        rout[0,:,:] = q
        rout[1,msk,:] = s
        return rout

class Mirror(Optic):
    def __init__(self, p=(0,0,0), n=(0,0,1), diam=1.):
        super().__init__(p, n, diam)
        
    def propagate(self, ray, clip=True):
        rout = np.full_like(ray, np.nan)
        
        q = self.intersect(ray, clip=clip)
        msk = ~np.isnan(q[:,0])
        s = ray[1,msk,:]
        
        sp = s - 2*np.dot(s, self.n)[:,None]*self.n[None,:]
        rout[0,:,:] = q
        rout[1,msk,:] = sp
        return rout

class Glass(Optic):
    def __init__(self, p=(0,0,0), n=(0,0,1), diam=1., n1=1., n2=1.):
        super().__init__(p, n, diam)
        self.n1 = n1
        self.n2 = n2
        self.nr = n1/n2
        
    def propagate(self, ray, indices=None, clip=True):
        #Propagate rays by transmission through  an index change, Snells law
        rout = np.full_like(ray, np.nan)
        
        q = self.intersect(ray, clip=clip)
        msk = ~np.isnan(q[:,0])
        s = ray[1,msk,:]
        #make sure there is always transmission and no reflection!
        #c = -np.dot(s, self.n)
        sn = np.dot(s, self.n)
        c = -sn
        #fudge for now to fix this, seems robust!
        f = np.where(sn>0,-1,1)
        
        dis = 1 - self.nr**2*(1 - c**2)
        #prevent total internal reflection
        msk[msk] = (dis>=0.)
        sp = self.nr*s + (f*self.nr*c - f*np.sqrt(dis))[:,None]*self.n[None,:]
        
        rout[0,:,:] = q
        rout[1,msk,:] = sp
        return rout

class CurvedMirror(Optic):
    def __init__(self, p=(0,0,0), n=(0,0,1), R=1., curv='CC', diam=25.4):
        super().__init__(p, n, diam)
        self.R = R
        x, dst = curved_offsets(self.R, self.r)
        self.intersect_d = dst
        if curv=='CC':
            self.cp = self.p + self.R*self.n
            self.poffs = self.p + self.n*x
        elif curv=='CX':
            self.cp = self.p - self.R*self.n
            self.poffs = self.p - self.n*x
        else:
            raise ValueError("Mirror type {} unknown! Curv has to be CC or CX".format(curv))
        self.curv = curv
    
    @staticmethod
    def curved_offsets(R, rapt):
        #calculates the on axis distance x and diag. distance dst for a curved mirror of finite apperture
        x =  R - np.sqrt(R**2 -  rapt**2)
        dst = np.sqrt(rapt**2+x**2)
        return x, dst    

    def plot(self, n_radii = 10, n_angles = 10, **kwargs):
        x, y = disc_coords(n_radii = n_radii, n_angles = n_angles, R=self.r)

        z = self.R-np.sqrt(self.R**2-x**2-y**2)
        if self.curv=='CX':
            z = -z
        indices = Triangulation(x,y).triangles.astype(np.uint32)
        mesh = k3d.mesh(np.vstack([x,y,z]).T, indices, **kwargs)
        t, r = self.transformation
        mesh.transform.rotation = r
        mesh.transform.translation = t
        return mesh
    
    def intersect(self, ray, clip=True):
        #do flat intersection first!
        q = self._intersect(ray, clip=clip, p=self.poffs)
        msk = ~np.isnan(q[:,0])
        r = ray[0,msk,:]
        s = ray[1,msk,:]

        d = self.cp - r
        ds = np.einsum("ji,ji->j",d,s)
        
        dis = ds**2 + self.R**2 - np.einsum("ji,ji->j",d,d)
        msk[msk] = dis>=0
        t12 = np.stack([ds+np.sqrt(dis), ds-np.sqrt(dis)], axis=0) #[2xNrays]
        #find the right intersection closer to the origin!
        x12 = r[None,:] + t12[:,:,None]*s[None,:] #[2xNraysx3]
        dist12 = np.linalg.norm(x12-self.p, axis=2) #[2xNrays]
        which = np.argmin(dist12, axis=0)
        x = x12[which,np.arange(x12.shape[1]),:]
        
        if clip:
            d = np.linalg.norm(x - self.p, axis=1)
            msk[msk] = (d<=self.intersect_d)
        
        xout = np.full_like(q, np.nan)
        xout[msk] = x
        return xout

        
    def propagate(self, ray, indices=None, clip=True):
        q = self.intersect(ray, clip=clip)
        msk = ~np.isnan(q[:,0])
        r = ray[0,msk,:]
        s = ray[1,msk,:]
        
        #find normal vectors
        n = self.cp - q[msk]
        nn = np.linalg.norm(n, axis=1)
        n *= 1./nn[:,None]
        
        sp = s - 2*np.einsum("ji,ji->j",s,n)[:,None]*n
        rout = np.full_like(ray, np.nan)
        rout[0,:,:] = q
        rout[1,msk,:] = sp
        return rout

class CurvedGlass(CurvedMirror):
    def __init__(self, p=(0,0,0), n=(0,0,1), R=1., curv='CC', diam=25.4, n1=1., n2=1.):
        super().__init__(p, n, R, curv, diam)
        self.n1 = n1
        self.n2 = n2
        self.nr = n1/n2
        
    def propagate(self, ray, indices=None, clip=True):
        #Propagate rays by transmission through  an index change, Snells law
        q = self.intersect(ray, clip=clip)
        msk = ~np.isnan(q[:,0])
        r = ray[0,msk,:]
        s = ray[1,msk,:]
        
        #find normal vectors
        n = self.cp - q[msk]
        nn = np.linalg.norm(n, axis=1)
        n *= 1./nn[:,None]
        if self.curv=='CX':
            n = -n
        
        #make sure there is always transmission and no reflection!
        #c = -np.einsum("ji,ji->j",s,n)
        sn = np.einsum("ji,ji->j",s,n)
        c = -sn
        #fudge for now to fix this, seems robust!
        f = np.where(sn>0,-1,1)
        
        dis = 1 - self.r**2*(1 - c**2)
        msk[msk] = (dis>=0.)
        sp = self.r*s + (f*self.r*c - f*np.sqrt(dis))[:,None]*n
        
        rout = np.full_like(ray, np.nan)
        rout[0,:,:] = q
        rout[1,msk,:] = sp
        return rout    

In [226]:
class Screen(Optic):
    def __init__(self, p=(0,0,0), n=(0,0,1), diam=25.4):
        super().__init__(p, n, diam)


In [140]:
def ray_bundle(p=(0,0,0), n=(0,0,1), n_radii=10, n_angles=2, R=1., divergence=0.):
    p = np.array(p)
    n = norm(n)
    radii = np.linspace(0., 1, n_radii)
    x, s = [[0,0,0],], [[0,0,1],]
    for r in radii[1:]:
        for t in np.linspace(0,2*np.pi,int(2*np.pi*r*n_angles)):
            x.append( [R*r*np.cos(t), R*r*np.sin(t), 0])
            s.append(norm([divergence*r*np.cos(t), divergence*r*np.sin(t), 1]))
       
    rays = np.stack([np.array(x), np.array(s)], axis=0)
    ez = np.array([0,0,1])
    R = getrot(ez, -n)
    rays2 = np.einsum("abi,ij->abj", rays, R)
    rays2[0,:,:] += p
    return rays2

def plot_rays(rays, plot, length=10.):
    s = k3d.vectors(origins=rays[0,...], vectors=rays[1,...], colors=[(0xff0000,)*rays.shape[1]*2], head_size=2.)
    plot += s
    ls = [k3d.line([rays[0,i,:], rays[0,i,:]+15*rays[1,i,:]]) for i in range(rays.shape[1]) if rays[0,i,0] is not np.nan]
    for l in ls:
        plot += l

## Benchmarks

In [165]:
rays = ray_bundle(n=(1,0,0),n_radii=5, n_angles=3, divergence=0*5e-3, R=0.5)

In [166]:
el = Optic(p=(5,0,0), n=(-1,0,0))

In [167]:
el = Mirror(p=(5,0,0), n=(-1,0,1))

In [168]:
el = CurvedMirror(p=(5,0,0), n=(-1,0,1), diam=1., R=5., curv='CC')

In [171]:
el = CurvedMirror(p=(5,0,0), n=(-1,0,1), diam=1., R=5., curv='CX')

In [199]:
el = Glass(p=(5,0,0), n=(-1,0,1), n2=1.5)

In [196]:
el = Glass(p=(5,0,0), n=(1,0,-1), n2=1.5)

In [203]:
el = CurvedGlass(p=(5,0,0), n=(-1,0,1), diam=1., R=5., curv='CC', n2=1.5)

In [206]:
el = CurvedGlass(p=(5,0,0), n=(-1,0,1), diam=1., R=5., curv='CX', n2=1.5)

In [209]:
el = CurvedGlass(p=(5,0,0), n=(1,0,-1), diam=1., R=5., curv='CX', n2=1.5)

In [210]:
rays2 = el.propagate(rays, clip=True)
#rays2

In [211]:
plot = k3d.plot(camera_auto_fit=True, antialias=True)

plot_rays(rays, plot)
plot_rays(rays2, plot)
plot += el.plot()
plot.display()

Output()

In [212]:
%load_ext line_profiler

In [217]:
%timeit el.propagate(rays, clip=True)

235 µs ± 4.34 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [299]:
%lprun -f Optic._intersect propagate_system(elements, rays)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:30: RuntimeWarning: invalid value encountered in greater
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:37: RuntimeWarning: invalid value encountered in greater


Timer unit: 1e-07 s

Total time: 0.0011067 s
File: <ipython-input-202-581deac965e8>
Function: _intersect at line 23

Line #      Hits         Time  Per Hit   % Time  Line Contents
    23                                               def _intersect(self, ray, clip=True, p=None):
    24                                                   #shape of Ray [2xNraysx3]
    25         7        813.0    116.1      7.3          x = np.full((ray.shape[1], 3), np.nan)
    26                                                   
    27         7        313.0     44.7      2.8          r, s = ray
    28         7        645.0     92.1      5.8          sn = s@self.n
    29                                                   #prevent ray perpendicular to surface
    30         7       2398.0    342.6     21.7          msk = np.abs(sn)>np.finfo(np.float32).eps
    31         7       2117.0    302.4     19.1          t = ((p - r[msk])@self.n)/sn[msk]
    32         7        941.0    134.4      8.5          x[m

In [218]:
def geometry(mir):
    
    def norm(arr, axis=-1):
        norm = np.sqrt(np.sum(arr**2, axis=-1))
        norm = np.expand_dims(norm, axis)
        return np.divide(arr, norm, where=(norm!=0))
    
    Nm = len(mir)
    M = mir[:,None,:]-mir[None,:,:]
    m = norm(M)
    
    n = norm(np.array([np.cross(m[j,j-1],m[j,(j+1)%Nm]) for j in range(Nm)]))
    refl = -norm(np.array([0.5*(m[j,j-1]+m[j,(j+1)%Nm]) for j in range(Nm)])) #vectors normal to reflecting mirrors
    angles = np.array([0.5*np.arccos(np.dot(m[j,j-1],m[j,(j+1)%Nm])) for j in range(Nm)])
    xin = n
    xout = n
    yin = norm(np.array([np.cross(n[j],m[j,j-1]) for j in range(Nm)]))
    yout = norm(np.array([np.cross(n[j],m[j,(j+1)%Nm]) for j in range(Nm)]))
    R = np.stack([np.array([[xout[i]@xin[(i+1)%Nm], yout[i]@xin[(i+1)%Nm]],\
                            [xout[i]@yin[(i+1)%Nm], yout[i]@yin[(i+1)%Nm]]]) for i in range(Nm)], axis=0)
    
    Ls = [np.linalg.norm(M[j-1,j]) for j in range(Nm)]
    Lrt = sum(Ls)

    return {'mir': mir, 'n': n, 'refl': refl, 'xin': xin, 'xout': xout, 'yin': yin, 'yout': yout, 'Ls': Ls, 'Lrt': Lrt}

def plot_geometry(geom, **kwargs):
    mir, n, refl, yin, yout = geom['mir'], geom['n'], geom['refl'], geom['yin'], geom['yout']
    Nm = len(mir)
    plot = k3d.plot(camera_auto_fit=True, antialias=True)

    col=0xff0000
    pf = 1.
    plt_line = k3d.line(pf*mir, shader='mesh', width=0.5, color=col)
    plt_line2 = k3d.line(pf*mir[(-1,0),...], shader='mesh', width=0.5, color=col)
    plot += plt_line
    plot += plt_line2
    plot += k3d.vectors(origins=pf*mir, vectors=n*2, use_head=True, head_size=3.)#Normals = xIn = xOut
    plot += k3d.vectors(origins=pf*mir, vectors=yin*2, use_head=True, head_size=3., color= 0xff8c00) #yIn
    plot += k3d.vectors(origins=pf*mir, vectors=yout*2, use_head=True, head_size=3., color= 0xff8c00) #yOut
    plot += k3d.vectors(origins=pf*mir, vectors=refl*2, use_head=True, head_size=3., color=0x00ff00)

    ey = np.array([0,1,0])
    ex = np.array([1,0,0])
    for i in range(Nm):
        mirror = Box(size=(1,10,10)).mesh

        mirror.transform.custom_matrix = pad3to4(getrot(ex, refl[i])) #get rotation matrix of mirror
        mirror.transform.translation = pf*mir[i]
        plot += mirror

    return plot

In [219]:
def SixMirror(dx=27.77, dy=8.0, dz=16.685, d=4.750, dzF=1.5825, Rfast=25.0):
    

    p1 = np.array([0,0,0])
    p2 = np.array([dx, dy, dzF])
    p3 = np.array([0, dy, dzF])
    p4 = np.array([dx, 2*dy, 0])
    p5 = np.array([d, dy+d, dz])
    p6 = np.array([dx-d, dy-d, dz])
    
    ps = np.stack([p1,p2,p3,p4,p5,p6], axis=0)
    geom = geometry(ps)
    ns = geom['refl']
    hi = 12.7
    qi=7.75
    elements = [CurvedMirror(p=p3, n=ns[2], diam=qi, R=Rfast, curv='CC'),\
                Mirror(p=p4, n=ns[3], diam=qi),\
                Mirror(p=p5, n=ns[4], diam=hi),\
                Mirror(p=p6, n=ns[5], diam=hi),\
                Mirror(p=p1, n=ns[0], diam=qi),\
                CurvedMirror(p=p2, n=ns[1], diam=qi, R=Rfast, curv='CC')]
    return elements, geom

In [250]:
elements, geom = SixMirror()

x0 = 0.5*(elements[0].p + elements[-1].p)
n0 = norm(elements[0].p - elements[-1].p)
n0[2] += 0.2
rs = ray_bundle(p=x0, n=n0, n_radii=5, n_angles=2, R=0.2)

screen = Screen(p=x0, n=-n0, diam=7.75)
elements.append(screen)

In [251]:
def propagate_system(elements, rays, Nrt=1, clip=True):
    rs = rays.copy()
    ind = np.arange(rs.shape[1])
    Nel = len(elements)
    #shape [Nel*Nrt+1,2,Nrays,3]
    allrays = np.empty((Nel*Nrt+1,*rays.shape))
    allrays[0,...] = rs
    for i in range(Nrt):
        for j, el in enumerate(elements):
            rs = el.propagate(rs, clip=clip)
            allrays[i*Nel+j+1,...] = rs
        
    return allrays

In [279]:
traj = propagate_system(elements, rs, Nrt=3)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:30: RuntimeWarning: invalid value encountered in greater
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:37: RuntimeWarning: invalid value encountered in greater


In [292]:
def clip_traj(traj):
    idx = np.where(np.isnan(traj[:,0]))[0]
    if len(idx)>0:
        idx = idx[0]
        return traj[:idx,:]
    else:
        return traj

def plot_trajs(trajs, plot, **kwargs):

    #ls = [k3d.line([rays[0,i,:], rays[0,i,:]+15*rays[1,i,:]]) for i in range(rays.shape[1]) if rays[0,i,0] is not np.nan]
    for i in range(trajs.shape[0]):
        t = clip_traj(traj[:,0,i,:])

        if t.shape[0]>1:
            l = k3d.line(t, **kwargs)
            plot += l

In [333]:
r0 = x0[None,:] + np.array([[0,0.2,0], [0,0,0.2], [0,0.2,0.2], [0,-0.2,-0.2]])

s0 = np.broadcast_to(n0, r0.shape)
ray0 = np.stack([np.atleast_2d(r0), np.atleast_2d(s0)], axis=0)

In [357]:
def find_eigenray(elements, ray0, lr = 0.05, maxiter=500, tol=1e-5, Nrt=1):
    rcur = ray0.copy()
    for i in range(maxiter):
        traj = propagate_system(elements, rcur, Nrt=Nrt)
        rnew = traj[-1,...]
        res = np.max(np.abs(rcur.flatten() - rnew.flatten()))

        rcur = (1.-lr)*rcur + lr*rnew
        if res<tol:
            break
    print("Finished in {} steps, reached tol {:.3e}".format(i, res))
    return rcur

def find_eigenray_animated(elements, ray0, lr = 0.05, maxiter=500, tol=1e-5, Nrt=1):
    rcur = ray0.copy()
    trajs = []
    for i in range(maxiter):
        traj = propagate_system(elements, rcur, Nrt=Nrt)
        rnew = traj[-1,...]
        trajs.append(traj[:,0,:,:])
        res = np.max(np.abs(rcur.flatten() - rnew.flatten()))
        rcur = (1.-lr)*rcur + lr*rnew
        if res<tol:
            break
    print("Finished in {} steps, reached tol {:.3e}".format(i, res))
    return rcur, np.stack(trajs, axis=0)

In [368]:
%timeit reig = find_eigenray(elements, ray0, lr=0.1, Nrt=1)

Finished in 218 steps, reached tol 9.351e-06
Finished in 218 steps, reached tol 9.351e-06
Finished in 218 steps, reached tol 9.351e-06
Finished in 218 steps, reached tol 9.351e-06
Finished in 218 steps, reached tol 9.351e-06
Finished in 218 steps, reached tol 9.351e-06
Finished in 218 steps, reached tol 9.351e-06
Finished in 218 steps, reached tol 9.351e-06
168 ms ± 9.24 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [364]:
reig, traj_anim = find_eigenray_animated(elements, ray0, lr=0.1, Nrt=1)

Finished in 218 steps, reached tol 9.351e-06


In [366]:
ts_dicts = [{str(t/100): traj_anim[t,:,r,:] for t in range(0, traj_anim.shape[0], 5)} for r in range(traj_anim.shape[2])]

In [367]:
plot = k3d.plot(camera_auto_fit=True, antialias=True)
plot += k3d.vectors(origins=x0, vectors=n0)

for el in elements:
    plot += el.plot(opacity=0.4)

#plot_trajs(traj, plot)
for ts in ts_dicts:
    plot += k3d.line(ts, shader='mesh', width=0.2, color=0x00ff00)

plot.display()

Output()